# Sampling and Energy Calculation of a Water Molecule

In this tutorial we explore how to sample the density of a water molecule and compute the total energy of the system using QMC. We first import all the module we will need in the tutorial

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from qmctorch.scf import Molecule
from qmctorch.wavefunction import SlaterJastrow
from qmctorch.sampler import Metropolis
from qmctorch.solver import Solver
from qmctorch.utils.plot_data import plot_walkers_traj

## Creating the system

We now create the `Molecule` object. We use here a water molecule and load the coordinates directly from a file. We also need to specify the quantum chemistry package from which we extract the atomic and molecular orbitals information. We choose here `pyscf` and secpify  a `sto-3g`  basis set. 

In [ ]:
# define the molecule
mol = Molecule(atom='water.xyz', unit='angs',
               calculator='pyscf', basis='sto-3g', name='water')

We can now define the wave function ansatz describing the electronic structure of the molecule. We use here a simple `SlaterJastrow` wave function and only include the ground state electronic configuration in the CI expansion. We use here the default Pade Jastrow form of the Jastrow factor. We therefore have a wave function of the type:

$$
\Psi(R) = J(R) D^0_\uparrow(r_\uparrow) D^0_\downarrow(r_\downarrow)
$$

In [ ]:
wf = SlaterJastrow(mol, configs='ground_state')

We now define the a Metropolis sampler, using only 100 walkers. Each walker contains here the positions of the 10 electrons of molecule. The electrons are initially localized around their atomic center, i.e. 8 around the oxygen atom and 1 around each hydrogen atom. We also specify here that the sampler will perform 500 steps with a step size of 0.25 bohr. 

In [ ]:
sampler = Metropolis(nwalkers=100, nstep=500, step_size=0.25,
                     nelec=wf.nelec, ndim=wf.ndim,
                     init=mol.domain('atomic'),
                     move={'type': 'all-elec', 'proba': 'normal'})

We can finally initialize the solver that will run the calculation

In [ ]:
solver = Solver(wf=wf, sampler=sampler)

## Sampling the density
We can use the wave function and the sampler we just defined to obtain sample of the electronic density. We can for example then plot the positions of the individual electrons in the plane of the molecule to vizually inspect the result of the sampling.

In [ ]:
pos = sampler(wf.pdf)
pos = pos.reshape(100,10,3).cpu().detach().numpy()
plt.scatter(pos[:,:,0],pos[:,:,1],s=0.5)

## Following indiviudal electron path
By default the sampler only record the position of the electrons at the very last step of the sampling process. We can however change and record all the positions of the electrons during the sampling to be able to track them.  

In [ ]:
sampler_singlewalker = Metropolis(nwalkers=1, nstep=500, step_size=0.25,
                     nelec=wf.nelec, ndim=wf.ndim,
                     ntherm=0, ndecor=1,
                     init=mol.domain('atomic'),
                     move={'type': 'all-elec', 'proba': 'normal'})

In [ ]:
pos = sampler_singlewalker(wf.pdf)
pos = pos.reshape(-1,10,3).detach().numpy()
plt.plot(pos[:,:,0], pos[:,:,1], marker="o", ls='--')

## Energy Calculation
To compute the energy of the system we first need to create a solver object that will handle all the orchestration of the calculation

In [ ]:
solver = Solver(wf=wf, sampler=sampler)

The energy can then be directly calculated

In [ ]:
obs = solver.single_point()

## Sampling Trajectory
We can also follow how the total energy thermalize during the sampling process. To this end we need to record the positions of the walkers during the sampling and not just at the end of it. We can then compute the local energies and the total energy at each recorded step of the trajectory. We can either create a new sampler or simply change the configuration of the sampler already included in our solver. We will put the number of thermalization steps to 0 and the number of decorellation step to 5.

In [ ]:
solver.sampler.ntherm = 0
solver.sampler.ndecor = 5

We can then resample the density and compute the local energy values along the sampling trajectory and finnaly plot it.

In [ ]:
pos = solver.sampler(solver.wf.pdf)
obs = solver.sampling_traj(pos)
plot_walkers_traj(obs.local_energy, walkers='mean')